In [14]:
import numpy as np
import matplotlib as plt
from scipy import optimize

# Pràctica 4: Espectres Òptics

## (A) Posada a punt de l'aparell 

_corresponents observacions_

## (B) Observació de l'espectre

_corresponents observacions_

## (C) Calibració de l'espectròmetre

### Mesures i dades

Mesures de la desviació mínima, $\delta_m$ per a cada longitud d'ona del mercuri

* `longs_Hg[i]` és la longitud d'ona de cada ratlla en Å
* `delta_Hg[i]` és la mitjana de les corresponents mesures de $\delta_m$, en graus

In [10]:
dades = [
    [4046.6, [35.95, 35.925]],
    [4078.2, [36.15, 36.21666]],
    [4358.3, [38.3, 38.3]],
    [4916.0, [41.04167, 41.1]],
    [5460.7, [42.783333, 42.75]],
    [5769.6, [43.45, 43.5]],
    [5789.7, [43.55, 43.5416666]]
]
delta_ref_raw = [110.0333, 110.05, 110.05]

In [9]:
longs_Hg = [dada[0] for dada in dades]

In [12]:
delta_Hg_raw = [dada[1] for dada in dades]
delta_Hg = [np.average(deltes) for deltes in delta_Hg_raw]
delta_ref = np.average(delta_ref_raw)

### Ajust de les constants del prisma
Fem l'ajust per a la fórmula de Hartmann

$$\lambda(\delta) = \lambda_0 + \frac{C}{\delta - \delta_0}$$

In [17]:
def hartmann(x, a, b, c):
    return a + b/(x - c)

params, params_err = optimize.curve_fit(hartmann, delta_Hg, longs_Hg)
params_err

array([[ 1.56423976e+03,  3.15127362e+04, -6.30354489e+00],
       [ 3.15127362e+04,  6.41963255e+05, -1.29380675e+02],
       [-6.30354489e+00, -1.29380675e+02,  2.62428633e-02]])

_faltarà fer l'ajust lineal per trobar R^2_

## (D) Determinació de longituds d'ona

Ara mesurem els angles de desviació mínima i calculem les longituds d'ona de les corresponents franges de l'espectre de cadmi fent servir la fórmula de Hartmann

* `longsCd[i]` és la longitud d'ona de cada ratlla en Å
* `deltaCd[i]` és la mitjana de les corresponents mesures de $\delta_m$, en graus 

In [ ]:
longsCd = []
longsCd_teo = [4413, 4799.9, 4678.2, 5155.1, 5085.8, 6438.5]
deltaCd = []

* Mesura de la ratlla violeta ($\lambda = 4413 Å$)

In [14]:
longsHg += []
deltaCd.append(mean([38.625, 38.6583333]))

* Mesura de la ratlla blava ($\lambda = 4678.2 Å$)

In [14]:
longsHg += []
deltaCd.append(mean([40.09166666, 40.091666666]))

* Mesura de la ratlla blava ($\lambda = 4799.9 Å$)

In [14]:
longsHg += []
deltaCd.append(mean([40.6, 40.625]))

* Mesura de la ratlla verda ($\lambda = 5085.8 Å$)

In [14]:
longsHg += []
deltaCd.append(mean([41.683333, 41.683333]))

* Mesura de la ratlla verda ($\lambda = 5155.1 Å$)

In [14]:
longsHg += []
deltaCd.append(mean([41.88333, 41.9083333]))

* Mesura de la ratlla vermella ($\lambda = 6438.5 Å$)

In [14]:
longsHg += []
deltaCd.append(mean([44.6, 44.65]))

_i successives mesures_

_s'han de calcular les longituds d'ona a partir de Hartmann i comparar-les amb les teòriques_

Fem l'anàlisi de l'error, fent servir
$$u(\lambda) = \frac{C}{(\delta - \delta_0)^2} u(\delta)$$
i la precisió en la mesura de l'angle, `u_delta`

In [15]:
u_delta = 0

## (E) Corba de dispersió del vidre
Fem servir la fórmula $$n(\lambda) = \frac{\sin{\left(\frac{\delta(\lambda) + \alpha}{2}\right)}}{\sin{\left(\frac{\alpha}{2}\right)}}$$
amb $\alpha = 60º$ per trobar els diversos índexs de dispersió del vidre

In [18]:
# Definim funcions trigonomètriques per graus
sind(x) = sin(180/pi * x)

In [20]:
index(x) = sin((x + 60)/2) / sin(60/2)
ns = []

# Calcular els corresponents índexos de refracció

_s'ha de calcular el nombre d'Abbe del vidre_

## (F) Observació de diferents espectres 

_corresponents observacions_